In [1]:
import os
import glob
import torch
import numpy as np
import pandas as pd
from PIL import Image

# Load Data + YOLO

In [6]:
raw_imgs_path = '../data/raw'
all_imgs = glob.glob(os.path.join(raw_imgs_path, '*.jpg'))

### DOWNSAMPLED FOR TESTING ###
# sample 10
# sampled_imgs = all_imgs[:10]

# # load imgs
#imgs = [Image.open(img_path) for img_path in sampled_imgs]

### THE REAL DEAL ###
# load imgs
imgs = [Image.open(img_path) for img_path in all_imgs]

# load yolo
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /home/npenoyer34/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-19 Python-3.10.13 torch-2.1.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
results = model(imgs)

# sanity check
# results.print()

dets = results.pandas().xyxy

# only keep dataframes and images with at least 2 subjects
filtered_dets = [det for det in dets if len(det) >= 2]
filtered_all_imgs = [img_path for img_path, det in zip(all_imgs, dets) if len(det) >= 2]

In [3]:
def crop_and_save(img_paths, detections_list, verbose=False):
  out_path = '../data/tightbox'
  os.makedirs(out_path, exist_ok=True)

  for img_path, detections in zip(img_paths, detections_list):
    # load img
    img = Image.open(img_path)
    # find the bounding box that contains all objects
    xmin = int(detections['xmin'].min())
    ymin = int(detections['ymin'].min())
    xmax = int(detections['xmax'].max())
    ymax = int(detections['ymax'].max())

    #find new area and see if it is sufficiently smaller than original area
    size=img.size[0]*img.size[1]
    cropped_size=(xmax-xmin)*(ymax-ymin)
    
    if  cropped_size < .75*size:


      #crop image around bounding box
      cropped_img = img.crop((xmin, ymin, xmax, ymax))

      # get image name; remove extension
      img_name = os.path.basename(img_path)
      img_name_no_ext = os.path.splitext(img_name)[0]

      # create new image name w/ bounding box coords
      new_img_name = f"{img_name_no_ext}_subject_box_{xmin}_{ymin}_{xmax}_{ymax}.jpg"

      # save cropped img
      save_path = os.path.join(out_path, new_img_name)
      cropped_img.save(save_path)

      if verbose:
        print(f"Saved: {save_path}")

In [ ]:
crop_and_save(filtered_all_imgs, filtered_dets, verbose=False)